In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Load the dataset
df = spark.read.csv("fraudTrain.csv", header=True, inferSchema=True)

# Show dataset structure
df.printSchema()
df.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: timestamp (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)

+---+---------------------+----------------+--------------------+-------------+------+---------+-----

In [36]:
from pyspark.sql.functions import count

# Count missing values
df.select([count(col(c)).alias(c) for c in df.columns]).show()

# Replace Null value with 0
df = df.fillna(0)

+-------+---------------------+-------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+-------+-------+---------+---------+---------+----------+--------+
|    _c0|trans_date_trans_time| cc_num|merchant|category|    amt|  first|   last| gender| street|   city|  state|    zip|    lat|   long|city_pop|    job|    dob|trans_num|unix_time|merch_lat|merch_long|is_fraud|
+-------+---------------------+-------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+-------+-------+---------+---------+---------+----------+--------+
|1296675|              1296675|1296675| 1296675| 1296675|1296675|1296675|1296675|1296675|1296675|1296675|1296675|1296675|1296675|1296675| 1296675|1296675|1296675|  1296675|  1296675|  1296675|   1296675| 1296675|
+-------+---------------------+-------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--

In [37]:
from pyspark.sql.functions import to_timestamp, hour, dayofweek

# Convert transaction date to timestamp
df = df.withColumn("trans_date_trans_time", to_timestamp(col("trans_date_trans_time"), "yyyy-MM-dd HH:mm:ss"))

# Extract time-based features
df = df.withColumn("hour", hour(col("trans_date_trans_time")))
df = df.withColumn("day_of_week", dayofweek(col("trans_date_trans_time")))

df.select("trans_date_trans_time", "hour", "day_of_week").show(5)

+---------------------+----+-----------+
|trans_date_trans_time|hour|day_of_week|
+---------------------+----+-----------+
|  2019-01-01 00:00:18|   0|          3|
|  2019-01-01 00:00:44|   0|          3|
|  2019-01-01 00:00:51|   0|          3|
|  2019-01-01 00:01:16|   0|          3|
|  2019-01-01 00:03:06|   0|          3|
+---------------------+----+-----------+
only showing top 5 rows



In [38]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as spark_sum, count as spark_count

# Define window by user and transaction date
window_spec = Window.partitionBy("cc_num", "trans_date_trans_time")

# Total daily spending per user
df = df.withColumn("daily_spending", spark_sum("amt").over(window_spec))

# Transaction count per day
df = df.withColumn("daily_transactions", spark_count("cc_num").over(window_spec))

df.select("cc_num", "amt", "daily_spending", "daily_transactions").show(5)

+-----------+------+--------------+------------------+
|     cc_num|   amt|daily_spending|daily_transactions|
+-----------+------+--------------+------------------+
|60416207185|117.11|        117.11|                 1|
|60416207185| 81.48|         81.48|                 1|
|60416207185| 52.47|         52.47|                 1|
|60416207185|  2.77|          2.77|                 1|
|60416207185|   8.5|           8.5|                 1|
+-----------+------+--------------+------------------+
only showing top 5 rows



In [39]:
from pyspark.sql.functions import radians, cos, sin, atan2, sqrt

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    return 2 * R * atan2(
        sqrt(sin((radians(lat2) - radians(lat1)) / 2) ** 2 +
             cos(radians(lat1)) * cos(radians(lat2)) *
             sin((radians(lon2) - radians(lon1)) / 2) ** 2),
        sqrt(1 - (sin((radians(lat2) - radians(lat1)) / 2) ** 2 +
                  cos(radians(lat1)) * cos(radians(lat2)) *
                  sin((radians(lon2) - radians(lon1)) / 2) ** 2))
    )

# Add a new column for distance
df = df.withColumn("distance", haversine(col("lat"), col("long"), col("merch_lat"), col("merch_long")))

df.select("lat", "long", "merch_lat", "merch_long", "distance").show(5)

+-------+---------+------------------+-----------+------------------+
|    lat|     long|         merch_lat| merch_long|          distance|
+-------+---------+------------------+-----------+------------------+
|36.0788| -81.1781|         36.011293| -82.048315|  78.5975684882306|
|48.8878|-118.2105|49.159046999999994|-118.186462|30.212175719210443|
|42.1808| -112.262|         43.150704|-112.154481|108.20608258720067|
|46.2306|-112.1138|         47.034331|-112.561071| 95.67323113819748|
|38.4207| -79.4629|         38.674999| -78.632459|  77.5567436258178|
+-------+---------+------------------+-----------+------------------+
only showing top 5 rows



In [40]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Convert category names to numerical index
indexer = StringIndexer(inputCol="category", outputCol="category_index")
df = indexer.fit(df).transform(df)

# One-hot encode category
encoder = OneHotEncoder(inputCol="category_index", outputCol="category_encoded")
df = encoder.fit(df).transform(df)

df.select("category", "category_index", "category_encoded").show(5)

+-------------+--------------+----------------+
|     category|category_index|category_encoded|
+-------------+--------------+----------------+
|     misc_net|          11.0| (13,[11],[1.0])|
|  grocery_pos|           1.0|  (13,[1],[1.0])|
|entertainment|           6.0|  (13,[6],[1.0])|
|gas_transport|           0.0|  (13,[0],[1.0])|
|     misc_pos|          10.0| (13,[10],[1.0])|
+-------------+--------------+----------------+
only showing top 5 rows



In [41]:
from pyspark.sql.functions import when

# Get fraud and non-fraud counts
fraud_df = df.filter(df.is_fraud == 1)
fraud_count = fraud_df.count()

# Calculate how many non-fraud cases we need for a 7:3 ratio
# If fraud is 30%, then non-fraud is 70%
target_non_fraud_count = int((7/3) * fraud_count)

# Sample non-fraud transactions to get the target count
non_fraud_df = df.filter(df.is_fraud == 0).sample(False, target_non_fraud_count / df.filter(df.is_fraud == 0).count())

# Combine balanced dataset
df = fraud_df.union(non_fraud_df)
# Check final distribution
df.groupBy("is_fraud").count().show()

+--------+-----+
|is_fraud|count|
+--------+-----+
|       1| 7506|
|       0|17532|
+--------+-----+



In [42]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.ml.linalg import Vectors

# Define a UDF to extract values from the vector
def extract_values(vector):
    return vector.toArray().tolist()

# Register the UDF
extract_values_udf = udf(extract_values, ArrayType(DoubleType()))

# Apply the UDF to the category_encoded column
df = df.withColumn("category_encoded_values", extract_values_udf(col("category_encoded")))

# Show the DataFrame to verify
df.select("category_encoded", "category_encoded_values").show(5, truncate=False)

+----------------+-----------------------------------------------------------------+
|category_encoded|category_encoded_values                                          |
+----------------+-----------------------------------------------------------------+
|(13,[1],[1.0])  |[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
|(13,[0],[1.0])  |[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
|(13,[1],[1.0])  |[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
|(13,[0],[1.0])  |[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
|(13,[1],[1.0])  |[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]|
+----------------+-----------------------------------------------------------------+
only showing top 5 rows



In [43]:
# Number of categories (replace with the actual number of categories in your dataset)
num_categories = 13

# Create separate columns for each category
for i in range(num_categories):
    df = df.withColumn(f"category_{i}", col("category_encoded_values").getItem(i))

# Drop the intermediate columns (if no longer needed)
df = df.drop("category_encoded", "category_encoded_values")

# Show the DataFrame to verify
df.show(5)

+------+---------------------+-----------+--------------------+-------------+-------+-------+------+------+--------------------+-------------+-----+-----+-------+---------+--------+--------------------+-------------------+--------------------+----------+------------------+-------------------+--------+----+-----------+--------------+------------------+------------------+--------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+
|   _c0|trans_date_trans_time|     cc_num|            merchant|     category|    amt|  first|  last|gender|              street|         city|state|  zip|    lat|     long|city_pop|                 job|                dob|           trans_num| unix_time|         merch_lat|         merch_long|is_fraud|hour|day_of_week|daily_spending|daily_transactions|          distance|category_index|category_0|category_1|category_2|category_3|category_4|category_5|category_6|ca

In [44]:
# Save the DataFrame as a single CSV file
df.coalesce(1).write.csv("train_processed_data.csv", header=True, mode="overwrite")